In [ ]:
| Product Name       | Variable | URL                                                                                     |
|-------------------|----------|-----------------------------------------------------------------------------------------|
| PACE_OCI_L2_AOP    | rrs      | [Link](https://www.earthdata.nasa.gov/data/catalog/ob-cloud-pace-oci-l2-aop-3.1#overview) |
| PACE_OCI_L2_BGC    | chlor_a  | [Link](https://www.earthdata.nasa.gov/data/catalog/ob-cloud-pace-oci-l2-bgc-3.1)        |
| PACE_OCI_L2_IOP    | bbp      | [Link](https://www.earthdata.nasa.gov/data/catalog/ob-cloud-pace-oci-l2-iop-3.1)        |


In [14]:
import earthaccess
import xarray as xr

earthaccess.login()

def load_pace_product(short_name, variable, lat_min, lon_min, lat_max, lon_max, start_date, end_date):
    results = earthaccess.search_data(
        short_name=short_name,
        bounding_box=(lon_min, lat_min, lon_max, lat_max),
        temporal=(start_date, end_date),
        count=-1
    )
    files = earthaccess.open(results)
    ds = xr.open_mfdataset(files, combine="by_coords")[[variable]]
    return ds
lat_min, lat_max = 36.621649, 36.623586
lon_min, lon_max = -121.904196, -121.9007891

ds_aop = load_pace_product("PACE_OCI_L2_AOP", "rrs", lat_min, lon_min, lat_max, lon_max, "2024-04-01", "2024-04-30")
ds_bgc = load_pace_product("PACE_OCI_L2_BGC", "chlor_a", lat_min, lon_min, lat_max, lon_max, "2024-04-01", "2024-04-30")
ds_iop = load_pace_product("PACE_OCI_L2_IOP", "bbp", lat_min, lon_min, lat_max, lon_max, "2024-04-01", "2024-04-30")
# not gridded, will crash :)

QUEUEING TASKS | :   0%|          | 0/39 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/39 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/39 [00:00<?, ?it/s]

ValueError: Could not find any dimension coordinates to use to order the Dataset objects for concatenation

ImportError: cannot import name 'EarthAccess' from 'earthaccess' (/srv/conda/envs/notebook/lib/python3.12/site-packages/earthaccess/__init__.py)